# Read raw text corpora

In [1]:
import os
import chardet
import magic 
import pandas as pd
from typing import Tuple, List, Iterable

def read_raw_corpus(files: Iterable[str], corpus: str) -> Tuple[pd.DataFrame, List[str]]:
    """
    Reads all files from file and creates a dataframe containing:
        * Raw text
        * title
        * year of publication
        * period label
        * genre label
        * region label
        * corpus label (has to set as param)
    
    NOTE: The Corpus label must be either 'innsbruck' or 'manchester'.
    If it is 'manchester' another split is used to read the metadata from the filenames.
    
    Also a list of tuples containing the files which could not be processed 
    and the error which was thrown while trying to do so is returned.
    
    :param file Liste of filepaths
    :param corpus corpus label string
    :return a dataframe + a list files which could not be processed
    """
    
    data = []
    errors = []
    for file in files:
    
        m_file = magic.Magic(mime_encoding=True)
        file_encoding = m_file.from_buffer(file)
        file = file.encode(file_encoding).decode('UTF-8')
        filename = os.path.basename(file)
        genre, period, region, year, title = filename.split('_')[:5]
        if corpus.lower() == 'manchester':
            if title.endswith('RAW.txt'):
                title = title.replace('RAW.txt', '')
            else:
                title = title.replace('.txt', '')

        with open(file, 'rb') as f:
            text_bytes = f.read()
            try:
                encoding_info = chardet.detect(text_bytes)
                text = text_bytes.decode(encoding_info['encoding'])
            except Exception as e:
                errors.append((file, str(e)))
                continue

        data.append({
            'filename': file,
            'text': text,
            'title': title,
            'genre': genre,
            'period': period,
            'region': region,
            'year': year,
            'filename': filename,
            'corpus': corpus
        })
        
    df = pd.DataFrame.from_dict(data)
    df = df.set_index('filename')
    return df, errors



ModuleNotFoundError: No module named 'magic'

# Read raw innsbruck corpus files into a dataframe

In [3]:
import glob

innsbruck_raw_files = glob.glob('GermInnC/**/*.txt')
print(len(innsbruck_raw_files))

638


In [ ]:
innsbruck_raw_df, errors = read_raw_corpus(innsbruck_raw_files, 'innsbruck')
print(errors)
print(innsbruck_raw_df.shape)
innsbruck_raw_df.head()

In [ ]:
innsbruck_raw_df.to_csv('innsbruck_dataset.csv', encoding='UTF-8')

# Read raw manchester corpus files into a dataframe

In [ ]:
manchester_raw_files = glob.glob('Manchester/RAW/*.txt')

In [ ]:
manchester_raw_df, errors = read_raw_corpus(manchester_raw_files, 'manchester')
print(errors)
print(manchester_raw_df.shape)
manchester_raw_df.head()

In [ ]:
manchester_raw_df.to_csv('manchester_dataset.csv', encoding='UTF-8')

# Concat raw dataframes

In [ ]:
full_raw_df = pd.concat((manchester_raw_df, innsbruck_raw_df))
print(full_raw_df.shape)
df.head()

In [ ]:
full_raw_df.to_csv('full_dataset.csv', encoding='UTF-8')

# Read tagged corpora

In [4]:
tagged_files = glob.glob('GermInnC Tagged/**/*.txt')

In [6]:
import os
from typing import Iterable
tagged_data = []
for file in tagged_files:
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    tokens = []
    pos_tags = []
    lemmas = []
    with open(file, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.rstrip()
            try:
                token, pos_tag, lemma = line.split('\t')
            except ValueError:
                continue
            tokens.append(token)
            pos_tags.append(pos_tag)
            lemmas.append(lemma)

    tagged_data.append({
        'tokens': " ".join(tokens),
        'pos_tags': " ".join(pos_tags),
        'lemmas': " ".join(lemmas),
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year,
        'corpus': None # TODO
    })

In [7]:
tagged_data[0]

{'tokens': 'Erster Aufzug . Erster Auftritt . Ion . Sieh , schon bepurpurt des Parnassus Gipfel Der Frühe Schein , der goldne Sonnenwagen Erhebt sich glorreich in die blaue Bahn ; Und kaum noch scheuchte den gesunden Schlaf Des Morgens frischer Hauch mir von den Wimpern : Drum eifrig an mein Werk , den heil ’ gen Dienst ! Vor allem aber muss ich dich begrüssen , Apollo , heitrer Gott ! der du von droben Das milde Licht herab zur Erde sendest , Und hier im Tempel mit der Weisheit Sprüchen Die dunkle Brust den Sterblichen erleuchtest . O küsse meine Stirn mit reinem Strahl , Du , den Gebieter ich und Vater nenne , Weil du im Heiligthum mich auferzogst , Auf dass ich , von der Menschen wüstem Treiben Ganz unberührt , der Jugend regen Trieb , Das frohe Leben deinem Dienste weihte . Hier saug ‘ ich deinen Athem in der Luft , Und es umfängt mich dieser Haine Schatten So zärtlich wie dein väterlicher Arm . Mir schweifen irr und unbestimmt die Wünsche Nicht in die Ferne : lass nur stets mich w

In [8]:
import pandas as pd

df = pd.DataFrame.from_dict(tagged_data)
df.to_csv('tagged_dataset.csv', encoding='UTF-8')

# Tagged Manchester

In [9]:
tagged_files_2 = glob.glob('Manchester/LING-COL/*.txt')

In [10]:
import os
from typing import Iterable
tagged_data_2 = []
for file in tagged_files_2:
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    tokens = []
    norms = []
    pos_tags = []
    lemmas = []
    with open(file, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.rstrip()
            try:
                pID, token, norm, pos_tag, lemma, moph, p, rest = line.split('\t')
                #token, pos_tag, lemma = line.split('\t')
            except ValueError:
                continue
            
            tokens.append(token)
            norms.append(norm)
            pos_tags.append(pos_tag)
            lemmas.append(lemma)

    tagged_data_2.append({
        'tokens': " ".join(tokens),
        #'norm': " ".join(norms),
        'pos_tags': " ".join(pos_tags),
        'lemmas': " ".join(lemmas),
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year
    })

In [11]:
tagged_data_2[0]

{'tokens': 'So koͤmmet euch der LoͤwRitter schwermuͤtig vor ? Ja ich kan nicht anders schliessen / weil er stets in tieffen Gedancken sitzet . Vielleicht ist er noch ermuͤdet von den vielen beschwerlichen Reisen und Abentheurlichen begebenheiten . Lasset ihn wol pflegen / damit er sich wieder erhole . Daran mangelt ihm nichts . Vielleicht ist er verliebet . Davor halte ich ihn gaͤntzlich : Den̄ er klaget immer / die schoͤnste Sonne der Welt habe ihren Glantz bißhero vor ihm verdeckt / und wenn er derselben nicht geniessen koͤnnen / so můsse er vor Leid vergehen . Habet ihr dann nicht mercken koͤnnen / was er durch seine Sonne verstehe ? Davon habe ich eigentlich nichts erfahren koͤnnen ; Aber in seinem Gemache henget eine guͤldene Sonne / in deren mitte ein kůnstliches L. von denen koͤstlichsten Diamanten versetzet ist . Das wird wol sein Ritterzeichen seyn . Davor halte ich es nicht ; denn ich zu mehrenmahlen ihn vor der Sonne auff den Knien sitzend und dieselbe anbetend gefunden . Ka

# Create the Manchester Tagged Dataframe and .csv

In [12]:
import pandas as pd
tagged_man_df = pd.DataFrame.from_dict(tagged_data_2)
tagged_man_df.shape

(336, 8)

In [13]:
tagged_man_df.to_csv('tagged_Mancheser_dataset.csv', encoding='UTF-8')

# Concat the tagged Datasets

In [15]:
full_tagged_df = pd.concat((tagged_man_df, df))
print(full_tagged_df.shape)
full_tagged_df

(974, 9)


C:\Users\janko\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,corpus,genre,lemmas,period,pos_tags,region,title,tokens,year
0,NaN,DRAM,so kommen ihr d Löwritter schwermütig vor ? ja...,P1,ADV VVFIN PPER ART NE ADJD PTKVZ SENT ITJ PPER...,NoD,Leonilda.txt,So koͤmmet euch der LoͤwRitter schwermuͤtig vo...,1673
1,NaN,DRAM,arm Herz / lassen ab zu hoffen / weil d glück ...,P1,"ADJA NN $, VVIMP PTKVZ PTKZU VVINF $, KOUS PDS...",NoD,Euridice.txt,ARmes herz / laß ab zu hoffen / weil das gluͤc...,1699
2,NaN,DRAM,ach / ich sein schon so müde / d ich kaum mehr...,P1,"ITJ $, PPER VAFIN ADV ADV ADJD $, PRELS PPER A...",NoD,Freyheit.txt,ACh / ich bin schon so müde / das ich kaum meh...,1700
3,NaN,DRAM,so sein ! er lassen ich hoch und überhoch bela...,P1,ADV VAFIN SENT PPER VVFIN PPER ADJD KON NN VVI...,OMD,Cardenio.txt,So ists ! er ließ mich hoch vnd uͤberhoch bela...,1657
4,NaN,DRAM,vertraute Charmium / dies sein -- Lebens-Höle ...,P1,"NN NN $, PDS VAFIN FM NN SENT PDS VAFIN ART NN...",OMD,Cleopatra.txt,VErtraute Charmium / dis ist di Lebens-Hoͤle ;...,1661
...,...,...,...,...,...,...,...,...,...
633,None,SERM,Kamerad ! daheim in die Silvester = Gottesdien...,P6,NN $. ADV APPR ART NN $( NN VVFIN PPER ADV NN ...,WMD,Zwölf,Kameraden ! Daheim in den Silvester = Gottesdi...,1917
634,None,SERM,eine jede Uebergang in+die Leben haben sein be...,P6,"ART PIAT NN APPRART NN VAFIN PPOSAT ADJA $, AD...",WOD,Hochzeits-Predigt,Ein jeder Uebergang im Leben hat seine besonde...,1906
635,None,SERM,"andächtig Christ , Geliebte in+die Herr ! als ...",P6,"ADJA NN $, NN APPRART NN $. KOUS ART ADJA NE P...",WOD,Der,"Andächtige Christen , Geliebte im Herrn ! Als ...",1914
636,None,SERM,darum wollen wir mit getroster Zuversicht er b...,P6,"PAV VMFIN PPER APPR ADJA NN PPER VVINF $, KOUS...",WOD,Zur,Darum wollen wir mit getroster Zuversicht ihn ...,1916


In [16]:
full_tagged_df.to_csv('full_taggeddataset.csv', encoding='UTF-8')